In [ ]:
import ax
import torch
import botorch
import numpy as np
import pymc3 as pm
import holoviews as hv
import plotly.graph_objects as go
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from gpytorch.mlls import ExactMarginalLogLikelihood
from botorch.models import SingleTaskGP
from botorch.optim import optimize_acqf
from botorch.fit import fit_gpytorch_model
from botorch.acquisition import UpperConfidenceBound, ExpectedImprovement
from ax.modelbridge.cross_validation import cross_validate
from ax.plot.diagnostic import interact_cross_validation
from ax.plot.scatter import plot_fitted
from ax.plot.slice import plot_slice
from ax.plot.contour import interact_contour, plot_contour
from ax.utils.notebook.plotting import render, init_notebook_plotting

from IPython.display import Image, IFrame
from test_functions import Problem05, Problem07, Problem09, Ursem01, EggCreate

hv.extension('bokeh', 'matplotlib')
init_notebook_plotting()

np.random.seed(1234)

%matplotlib inline

In [ ]:
def kernel_function_plot(kernel,
                         n_points: int,
                         n_samples: int = 3, 
                         width: int = 400, 
                         height: int = 400):
    """Returns plots that describe the behavior of the kernel"""
    # Data
    xs = np.linspace(0, 1, n_points)[:,None]
    K = kernel(xs).eval()
    ys = K[0, :]
    samples = pm.MvNormal.dist(mu=np.zeros(K.shape[0]), cov=K).random(size=n_samples).T
    # plots
    similarity = hv.Curve((xs, ys), vdims=['similarity']).opts(
        toolbar=None, ylabel="k(0, x)", height=height, width=width)
    similarity = similarity.redim(similarity=hv.Dimension('similarity', range=(0, 1.1)))
    points = hv.Scatter((xs, ys), vdims=['similarity']).opts(color='r', marker='*', size=6, alpha=0.5)
    cov_matrix_plot = hv.Image(K, kdims=["x", "y"], vdims=['k']).opts(
        cmap='viridis', colorbar=True, xaxis=None, yaxis=None,
        tools=['hover'], ylabel="y", xlabel="x", height=height, width=width)
    
    curves = {}
    for sample in range(n_samples):
        c = hv.Curve((xs, samples[:, sample])).opts(height=height, width=width + 250)
        sample_points = hv.Scatter((xs, samples[:, sample])).opts(color='k', marker='*', alpha=0.3, size=3)
        curves[sample] = c * sample_points

    nd_overlay = hv.NdOverlay(curves, kdims=['samples']).opts(toolbar=None)
    nd_overlay = nd_overlay.opts(
        {'Curve': {'color': hv.Cycle('Category10')}}
    )
    plot = (
        (similarity * points) + cov_matrix_plot + nd_overlay
    ).opts(hv.opts.Layout(shared_axes=False, shared_datasource=False))
    return plot


def get_sobel_seq(n: int, lower_bound: float, upper_bound: float):
    """Generates a Sobol sequence o n points"""
    if lower_bound > upper_bound:
        raise ValueError('upper_bound can not be smaller than lower_bound')
    sobol = ax.Models.SOBOL(
        search_space=ax.SearchSpace(
            parameters=[
                ax.RangeParameter(
                    name="x", 
                    lower=lower_bound, 
                    upper=upper_bound,
                    parameter_type=ax.ParameterType.FLOAT, 
                ),
            ]
        )
    )
    sobol_seq = [arm.parameters['x'] for arm in sobol.gen(N_INIT).arms]
    return sobol_seq

# Introduction to Bayesian optimization

__Slides__:

## Raúl Peralta Lozada

* Blog: https://raulpl.github.io/
* Twitter: @tnak25
* email: raulpl25@gmail.com
* OPI Analytics

<img src="./figures/OPI_logo.jpeg" alt="opi_logo" title="opi_logo" width="600" height="50" />
<img src="./figures/OPI_team.jpg" alt="opi" title="opi" width="1000" height="300" />

# Objectives:

1. Understand the general concepts behind Bayesian optimization
2. Use Python to solve this kind of optimization problems

## Optimization 
Find the global maximizer (or minimizer) of an objective function $f$ in some design space $\mathcal{X}$

$$x^{*} = argmax_{x \in \mathcal{X}} f(x)$$

In [ ]:
hv.Store.current_backend = 'matplotlib'
xs = np.linspace(-3, 3, 100)
ys = - np.power(xs, 2)
parabola = (
    hv.Curve((xs, ys)).opts(title=r"$ y=-x^2 $", linewidth=3) * 
    hv.VLine(0, label='h').opts(color='red') *
    hv.Labels(([1.5], [0], r'$\frac{dy}{dx}=-2x=0$')).opts(size=13)
).opts(fig_size=200, fontsize={'labels': 14})
parabola.redim(y=hv.Dimension('y', range=(-8, 1)))

## Problems in real world applications:

* Black box functions
* Noisy evaluations
* Restricted evaluation budget
* Unknown gradient information

### Examples:

* A/B testing
* Hyperparameter tunning
* Resource extraction
* Robotics

In [ ]:
problem_unknown = Problem05()
points_unknown = [(0.5, problem_unknown(0.5)),]

def interactive_optimization(x, y):
    if x:
        y = problem_unknown(x) + float(np.random.normal(scale=0.2, size=1))
        points_unknown.append((x, y))
    points_plot = hv.Points((points_unknown)).opts(color='k', marker='*', size=12, width=900, height=600)
    points_plot = points_plot.redim(
        x=hv.Dimension('x', range=(problem_unknown.bounds[0], problem_unknown.bounds[1])),
        y=hv.Dimension('y', range=(-2.2, 2.2))
    )
    plot = points_plot
    return plot

In [ ]:
# hv.Store.current_backend = 'bokeh'
# tap = hv.streams.Tap(transient=False)
# dmap = hv.DynamicMap(interactive_optimization, streams=[tap]).opts(tools=['hover', 'tap']).opts(framewise=True)
# dmap

In [ ]:
xs = np.linspace(*problem_unknown.bounds, num=200)
ys = np.array([problem_unknown(x) for x in xs])
plot = (
    hv.Curve((xs, ys)) * 
    hv.Scatter(points_unknown).opts(marker='*', size=10, color='k')
).opts(width=900, height=600)
plot

# Exploration-exploitation dilemma


__Exploration__: collect information about the domain

__Exploitation__: make the best decision given the current information

<sup>http://www0.cs.ucl.ac.uk/staff/d.silver/web/Teaching_files/XX.pdf<sup>

# Bayesian optimization

Bayesian optimization is a sequential design strategy for global optimization of black-box functions.

### Key components:
* *__Statistical model / surrogate model__*: captures the current belief about of the latent function $f$. 
* *__Acquisition function__*: evaluates the utility of candidate points for the next evaluation of $f$. This function determines the exploration strategy.

## Algorithm
* Unknown noisy objective function $f$
* Input domain $\mathcal{X}$
* Fixed query budget $T$
* Initial sample (optional) $D$

for n = 1, 2, ... $T$, do
1. Select new $x_{n + 1}$ by optimizing __acquisition function__ $\alpha$  
    $x_{n + 1} = argmax_{x \in \mathcal{X}} \alpha(x; D_n)$
    
2. Query objective function $f$ to obtain $y_{n + 1}$
3. Augment data $D_{n + 1} = \{ D_{n}, (x_{n + 1}, y_{n + 1}) \}$
4. Update __statistical model__

<sub>[Taking the Human Out of the Loop: A Review of Bayesian Optimization](https://ieeexplore.ieee.org/document/7352306)<sub>

# Statistical model

We need a model that can:
* __Learn__ from previous observations
* __Predict__ the value of new points
* Quantify the level of __uncertainty__ of those predictions

# Gaussian process (GP)

A GP is a collection of random variables, any finite number of which have a joint Gaussian distribution. This model is used to describe a distribution over functions. GPs are completely specified by:
1. A mean function $\mu(x)$ that is the average of all functions.
2. A covariance function or kernel $k(x, x')$ that describes our assumptions over the structure of the possible set of functions.


In [ ]:
IFrame('https://distill.pub/2019/visual-exploration-gaussian-processes/', width=1400, height=700)

In [ ]:
# Multivariate Gaussian distribution
mean = [0, 0]
cov = [
    (1, -0.9),
    (-0.9, 1)
]
mv_gaussian_sample = np.random.multivariate_normal(mean, cov, 1_000)
hv.Points(mv_gaussian_sample, kdims=['x1', 'x2']).opts(alpha=0.4, size=5, width=500, height=500)

## Squared Exponential / Exponentiated Quadratic:  
* $k(x, x') = \mathrm{exp}\left[ -\frac{(x - x')^2}{2 \ell^2} \right]$

In [ ]:
# Example taken from:
# https://colab.research.google.com/github/fonnesbeck/Bios8366/blob/master/notebooks/Section5_1-Gaussian-Processes.ipynb
hv.Store.current_backend = 'bokeh'
ls = 1
kernel = pm.gp.cov.ExpQuad(1, ls=ls)
kernel_function_plot(kernel, 100)

## Matern 3/2
* $k(x, x') = \left(1 + \frac{\sqrt{3(x - x')^2}}{\ell}\right) \mathrm{exp}\left[ - \frac{\sqrt{3(x - x')^2}}{\ell} \right]$

In [ ]:
ls = 0.1
kernel = pm.gp.cov.Matern32(1, ls=ls)
kernel_function_plot(kernel, 100)


## Periodic
 * $k(x, x') = \exp\left( -\frac{2 \sin^{2}(\pi |x - x'|\frac{1}{T})}{\ell^2}     \right)$

In [ ]:
ls = 0.6
period = 0.3
kernel = pm.gp.cov.Periodic(1, ls=ls, period=period)
kernel_function_plot(kernel, 150)


## Kernel composition
 
Locally periodic = __Periodic__ $\times$ __Squared Exponential__


In [ ]:
ls1 = 0.6
period = 0.3
ls2 = 0.3
kernel = pm.gp.cov.Periodic(1, ls=ls1, period=period) * pm.gp.cov.ExpQuad(1, ls=ls2)
kernel_function_plot(kernel, 100)

In [ ]:
def mean_cov_plot(x_train, y_train, x_test, trace, GP):
    mu, cov = GP.predict(x_test, point=trace[-1], diag=True)
    sd = np.sqrt(cov)
    sd_region = hv.Area((x_test, mu - 2 * sd, mu + 2 * sd), 
                        vdims=['lower', 'upper'], label='±2sd(𝑥)').opts(alpha=0.2)
    mean_curve = hv.Curve((x_test, mu), label='𝜇(𝑥)').opts(color='red')
    train_points = hv.Scatter(
        (x_train, y_train), 
        label='observed').opts(color='k', marker='*', size=8, legend_position='bottom_left')
    plot = (mean_curve * sd_region * train_points).opts(width=600, height=380, toolbar=None)
    return plot


def sample_model(model: pm.Model, trace, GP, test_x):
    with model:
        f_pred = GP.conditional('f_pred', test_x)
        pred_samples = pm.sample_posterior_predictive(trace, vars=[f_pred], samples=100)
    return pred_samples['f_pred']


def samples_plot(x_train, y_train, x_test, samples):
    train_points = hv.Scatter((x_train, y_train)).opts(color='k', marker='*', size=8)
    curves = hv.Path((x_test, samples.T)).opts(alpha=0.3, color='orange')
    plot = (curves * train_points).opts(width=600, toolbar=None)
    return plot


def function_plot(xs, ys, train_x, train_y):
    curve = hv.Curve((xs, ys)).opts(width=600, height=380, toolbar=None)
    train_points = hv.Scatter(
        (train_x, train_y), label='observed'
    ).opts(color='k', marker='*', size=8, toolbar=None, legend_position='bottom_left')
    plot = curve * train_points
    return plot



In [ ]:
N_POINTS: int = 150
N_INIT: int = 16
problem = Problem09()
xs = np.linspace(*problem.bounds, num=N_POINTS)
ys = np.array([problem(x) for x in xs])

minmax_scaler = MinMaxScaler().fit(xs[:, np.newaxis])
standar_scaler = StandardScaler().fit(ys[:, np.newaxis])

xs_normalized = minmax_scaler.transform(xs[:, np.newaxis])
ys_standardized = standar_scaler.transform(ys[:, np.newaxis])

sobol_seq = get_sobel_seq(N_INIT, problem.bounds[0], problem.bounds[1])
train_x = np.array(sobol_seq)

train_y = np.array([problem(x) for x in train_x])
train_y = train_y + np.random.normal(scale=0.2, size=N_INIT)
train_x = minmax_scaler.transform(train_x[:, None])
train_y = standar_scaler.transform(train_y[:, None])

train_iterations = {}
for i in range(N_INIT):
    train_x_i = train_x[0:i + 1]
    train_y_i = train_y[0:i + 1]
    train_iterations[i] = (train_x_i, train_y_i)

In [ ]:
def train_pymc3_gp(train_x, train_y):
    # Model definition
    with pm.Model() as model:
        ls = pm.Gamma('ls', alpha=2, beta=9)
        e = pm.Gamma('e', alpha=2, beta=9)
        cov = pm.gp.cov.ExpQuad(1, ls=ls)
        GP = pm.gp.Marginal(cov_func=cov)
        y_pred = GP.marginal_likelihood('y_pred', X=train_x, y=train_y, noise=e)
        prior = pm.sample_prior_predictive()
        trace = pm.sample(1_200, tune=1_000)
        posterior_predictive = pm.sample_posterior_predictive(trace)
    return model, trace, GP


train_plots = {}
for i, (train_x, train_y) in train_iterations.items():
    print(f'Iteration: {i}')
    model, trace, GP = train_pymc3_gp(train_x, train_y.flatten())
    plot = (
        function_plot(xs_normalized, ys_standardized, train_x, train_y) + 
        mean_cov_plot(train_x, train_y, xs_normalized, trace, GP)
    ).cols(1)
    train_plots[i] = plot

In [ ]:
model, trace, GP = train_pymc3_gp(train_x, train_y.flatten())
f_samples = sample_model(model, trace, GP, xs_normalized)

## GP training

In [ ]:
hmap = hv.HoloMap(train_plots, kdims=['i']).opts(framewise=True).collate()
hmap

## Posterior samples

In [ ]:
(samples_plot(train_x, train_y, xs_normalized, f_samples) * function_plot(xs_normalized, ys_standardized, train_x, train_y)).opts(width=800, height=500)

# Acquisition functions

Acquisition functions are heuristics employed to evaluate the usefulness of one of more design points for achieving the objective of maximizing the underlying black box function<sup>1</sup> . The predictions and the uncertainty estimates are combined to derive the explorarion strategies. Common acquisition functions are:

* Upper confidence bound
* Thompson sampling
* Probability of improvement
* Expected improvement
* Predictive entropy search

<sup>1</sup>[https://botorch.org/docs/acquisition](https://botorch.org/docs/acquisition)

In [ ]:
Image('./figures/overview_bayesopt.svg.png', width=1500)

# Upper Confidence Bound (UCB)

*Optimism in the face of uncertainty*

UCB is a function that trades off between greedy maximization and uncertainty reduction by taking into account the current value of the mean function and the value of the (weighted) standard deviation for each point. $\beta$ controls the explore-exploit trade-off.

$$\alpha_{UCB}(x; D_n) = \mu_{n}(x) + \beta_{n} \sqrt{\sigma_{n}(x)}$$ 
$$\beta_{n} \geqslant 0$$

In [ ]:
def predict_mean_and_var(model, test_x):
    model.eval()
    with torch.no_grad():
        f_preds = model(test_x)
        f_mean = f_preds.mean
        f_var = f_preds.variance
    return f_mean.numpy(), f_var.numpy()


def mean_sd_plot(mean_pred, var_pred, test_x):
    sd = np.sqrt(var_pred)
    sd_region = hv.Area((test_x, mean_pred - 2 * sd, mean_pred + 2 * sd),
                        vdims=['lower', 'upper'], label='±2sd(𝑥)').opts(alpha=0.2, toolbar=None)
    mean_curve = hv.Curve((test_x, mean_pred), label='𝜇(𝑥)').opts(color='red', width=600, height=380)
    plot = (mean_curve * sd_region)
    return plot
    

def acquisition_function_eval(acq_f, test_x):
    acq_f.eval()
    with torch.no_grad():
        evaluations = acq_f(test_x.unsqueeze(2))
    return evaluations.numpy()


def acquisition_function_view(test_x, acq_func_evals, width=600, height=380):
    best_point = float(test_x[acq_func_evals.argmax()])
    acq_min, acq_max = float(acq_func_evals.min()), float(acq_func_evals.max())
    curve = hv.Curve(
        (test_x, acq_func_evals), vdims=['alpha']
    ).opts(width=width, height=height, toolbar=None)
    curve = curve.redim(alpha=hv.Dimension('alpha', range=(acq_min - 0.1, acq_max + 0.1)))
    vline = hv.VLine(best_point).opts(color='red', line_width=4)
    return curve * vline


def training_data_view(train_x, train_y):
    "Scatter plot of the training data"
    train_points_scatter = hv.Scatter(
        (train_x, train_y), label='observed'
    ).opts(color='k', marker='*', size=8, legend_position='bottom_left')
    return train_points_scatter

In [ ]:
# Setup Upper Confidence Bound

N_INIT = 5
N_POINTS = 500
N_TRIALS: int = 21
NOISE_SCALE = 0.2
BETA = 20
problem = Problem09()


xs = np.linspace(*problem.bounds, num=N_POINTS)
ys = np.array([problem(x) for x in xs])

minmax_scaler = MinMaxScaler().fit(xs[:, np.newaxis])
standar_scaler = StandardScaler().fit(ys[:, np.newaxis])

xs_normalized = minmax_scaler.transform(xs[:, np.newaxis])
ys_standardized = standar_scaler.transform(ys[:, np.newaxis])

sobol_seq = get_sobel_seq(N_INIT, problem.bounds[0], problem.bounds[1])

xs_init = np.array(sobol_seq)
ys_init = np.array([problem(x) for x in xs_init])
ys_init = ys_init + np.random.normal(scale=NOISE_SCALE, size=N_INIT)
xs_init = minmax_scaler.transform(xs_init[:, None])
ys_init = standar_scaler.transform(ys_init[:, None])
train_x = torch.FloatTensor(xs_init)
train_y = torch.FloatTensor(ys_init)

# Surrogate model (GP)
gp = SingleTaskGP(train_x, train_y)

# marginal log likelihood (loss function)
mll = ExactMarginalLogLikelihood(gp.likelihood, gp)
mll = fit_gpytorch_model(mll, max_retries=100)

test_points = torch.FloatTensor(xs_normalized)
bounds = torch.stack([torch.zeros(1), torch.ones(1)])

plots = {}
for i in range(N_TRIALS):
    gp = SingleTaskGP(train_x, train_y)
    mll = ExactMarginalLogLikelihood(gp.likelihood, gp)
    mll = fit_gpytorch_model(mll, max_retries=50)
    UCB = UpperConfidenceBound(gp, beta=BETA, maximize=True)
    new_x, _ = optimize_acqf(
        UCB, bounds=bounds, q=1, num_restarts=100, raw_samples=100,
    )
    new_x_unscaled = minmax_scaler.inverse_transform(new_x.numpy())
    new_y = np.array(problem(float(new_x_unscaled[0][0])))
    # add noise
    new_y = new_y + np.random.normal(scale=NOISE_SCALE, size=1)
    new_y = standar_scaler.transform(new_y.reshape((1,1)))
    new_y = torch.FloatTensor(new_y)
    # Save evaluations
    f_mean, f_var = predict_mean_and_var(gp, test_points)
    ucb_evals = acquisition_function_eval(UCB, test_points)
    layout = (
        mean_sd_plot(f_mean, f_var, test_points.numpy()) * 
        training_data_view(train_x.numpy(), train_y.numpy()) 
    )
    layout = (layout + acquisition_function_view(test_points.numpy(), ucb_evals)).cols(1)
    plots[i + N_INIT] = layout
    # Update training 
    train_x = torch.cat([train_x, new_x])
    train_y = torch.cat([train_y, new_y])

In [ ]:
# Upper Confidence Bound
hmap = hv.HoloMap(plots, kdims=['n_points']).opts(framewise=True).collate()
hmap

# Expected improvement (EI)

EI is one of the most commonly used acquisition due to its practical performance and its analytical solution under the GP model.


$$\text{EI}(x) = \mathbb{E}\bigl[\max(f(x) - f^*, 0)\bigr] = \int \max(f(x) - f^*, 0) p(f(x)|x, D) df$$

<sup>[A Tutorial on Bayesian Optimization of Expensive Cost Functions, with Application to Active User Modeling and Hierarchical Reinforcement Learning](https://arxiv.org/abs/1012.2599)<sup>

In [ ]:
Image('./figures/probanility_of_improvement.png', width=800, height=300)

In [ ]:
# Setup Expected improvement

N_INIT = 5
N_POINTS = 500
NOISE_SCALE = 0.2
problem = Problem09()

xs = np.linspace(*problem.bounds, num=N_POINTS)
ys = np.array([problem(x) for x in xs])

minmax_scaler = MinMaxScaler().fit(xs[:, np.newaxis])
standar_scaler = StandardScaler().fit(ys[:, np.newaxis])

xs_normalized = minmax_scaler.transform(xs[:, np.newaxis])
ys_standardized = standar_scaler.transform(ys[:, np.newaxis])

sobol_seq = get_sobel_seq(N_INIT, problem.bounds[0], problem.bounds[1])

xs_init = np.array(sobol_seq)
ys_init = np.array([problem(x) for x in xs_init])
ys_init = ys_init + np.random.normal(scale=NOISE_SCALE, size=N_INIT)
xs_init = minmax_scaler.transform(xs_init[:, None])
ys_init = standar_scaler.transform(ys_init[:, None])
train_x = torch.FloatTensor(xs_init)
train_y = torch.FloatTensor(ys_init)

# Surrogate model (GP)
gp = SingleTaskGP(train_x, train_y)

# marginal log likelihood (loss function)
mll = ExactMarginalLogLikelihood(gp.likelihood, gp)
mll = fit_gpytorch_model(mll, max_retries=100)

N_TRIALS: int = 21
test_points = torch.FloatTensor(xs_normalized)
bounds = torch.stack([torch.zeros(1), torch.ones(1)])

plots = {}

for i in range(N_TRIALS):
    gp = SingleTaskGP(train_x, train_y)
    mll = ExactMarginalLogLikelihood(gp.likelihood, gp)
    mll = fit_gpytorch_model(mll, max_retries=50)
    # Take the current best value
    best_f = train_y.max()
    EI = ExpectedImprovement(gp, best_f=best_f, maximize=True)
    new_x, _ = optimize_acqf(
        EI, bounds=bounds, q=1, num_restarts=100, raw_samples=100
    )
    new_x_unscaled = minmax_scaler.inverse_transform(new_x.numpy())
    new_y = np.array(problem(float(new_x_unscaled[0][0])))
    # add noise
    new_y = new_y + np.random.normal(scale=NOISE_SCALE, size=1)
    new_y = standar_scaler.transform(new_y.reshape((1,1)))
    new_y = torch.FloatTensor(new_y)
    # Save evaluations
    f_mean, f_var = predict_mean_and_var(gp, test_points)
    ei_evals = acquisition_function_eval(EI, test_points)
    layout = (
        mean_sd_plot(f_mean, f_var, test_points.numpy()) * 
        training_data_view(train_x.numpy(), train_y.numpy()) 
    )
    layout = (layout + acquisition_function_view(test_points.numpy(), ei_evals)).cols(1)
    plots[i + N_INIT] = layout
    # Update training 
    train_x = torch.cat([train_x, new_x])
    train_y = torch.cat([train_y, new_y])

In [ ]:
# Expected Improvement
hmap = hv.HoloMap(plots, kdims=['n_points']).opts(framewise=True).collate()
hmap

<img src="./figures/ax_botorch.jpg" alt="botorch" title="Botorch" width="650" height="500" />

*__Botorch__*: is a library for Bayesian optimization research built on top of __PyTorch__ and __GPyTorch__.

*__Ax__*: is an adaptive experimentation platform. It provides easy-to-use APIs to interface with BoTorch, along with the management necessary for production-ready services and reproducible research. This allows developers to focus on the applied problems, such as exploring configurations and understanding trade-offs between objectives.

<sub>[Open-sourcing Ax and BoTorch: New AI tools for adaptive experimentation](https://ai.facebook.com/blog/open-sourcing-ax-and-botorch-new-ai-tools-for-adaptive-experimentation/)<sub>

In [ ]:
Image('./figures/botorch_ax_diagram.jpg')

In [ ]:
POINTS = 100
problem = EggCreate()
points = np.linspace(problem.bounds[0][0], problem.bounds[1][0], POINTS)
zs = np.empty((POINTS, POINTS))
for x in range(POINTS):
    for y in (range(POINTS)):
        zs[y, x] = problem(points[x], points[y])
fig = go.Figure(data=[go.Surface(z=zs, colorscale='Viridis')])
fig.update_layout(title=f'{problem.__class__.__name__}', autosize=False, width=1_000, height=600)
fig.show()

In [ ]:
N_INIT: int = 5
N_TRIALS: int = 50
NOISE_SCALE = 0.1
objective_name = f'{problem.__class__.__name__}'

# Search space
search_space = ax.SearchSpace(
    parameters=[
        ax.RangeParameter(
            name='x', parameter_type=ax.ParameterType.FLOAT, 
            lower=problem.bounds[0][0], upper=problem.bounds[1][0]
        ),
        ax.RangeParameter(
            name='y', parameter_type=ax.ParameterType.FLOAT,
            lower=problem.bounds[0][1], upper=problem.bounds[1][1]
        ),
    ]
)

# Experiment object
experiment = ax.SimpleExperiment(
    name=f'test_{objective_name}',
    search_space=search_space,
    evaluation_function=lambda p: problem(p["x"], p["y"]) + float(np.random.normal(scale=NOISE_SCALE, size=1)),
    objective_name=objective_name,
    minimize=False,
)

# Initial samples
sobol = ax.Models.SOBOL(experiment.search_space)
for i in range(N_INIT):
    experiment.new_trial(generator_run=sobol.gen(1))

# Bayesian optmization loop
best_arm = None
for i in range(N_TRIALS):
    gpei = ax.Models.GPEI(experiment=experiment, data=experiment.eval())
    generator_run = gpei.gen(1)
    best_arm, _ = generator_run.best_arm_predictions
    experiment.new_trial(generator_run=generator_run)
    
model = ax.Models.GPEI(experiment=experiment, data=experiment.eval())
best_parameters = best_arm.parameters
best_parameters

In [ ]:
render(plot_contour(model=model, param_x="x", param_y="y", metric_name=objective_name))

In [ ]:
cv_results = cross_validate(model)
render(interact_cross_validation(cv_results))

In [ ]:
render(plot_slice(model, "x", objective_name, slice_values=best_parameters))

In [ ]:
render(plot_slice(model, "y", objective_name, slice_values=best_parameters))

# Summary

* Explore-exploit trade-off.
* Bayesian optimization is a technique that can help us find the best parameters of a function in a systematic manner.
* Careful choice of statistical model is more important than the choice of an acquisition function.
* Python has libraries that can help you solve this kind of problems

# Resources

* [Taking the Human Out of the Loop: A Review of Bayesian Optimization](https://ieeexplore.ieee.org/document/7352306)
* [A Tutorial on Bayesian Optimization of Expensive Cost Functions, with Application to Active User Modeling and Hierarchical Reinforcement Learning](https://arxiv.org/abs/1012.2599)
* [Bayesian optimization UAI 2018](https://www.youtube.com/watch?v=C5nqEHpdyoE)
* [Open-sourcing Ax and BoTorch: New AI tools for adaptive experimentation](https://ai.facebook.com/blog/open-sourcing-ax-and-botorch-new-ai-tools-for-adaptive-experimentation/)
* [A Visual Exploration of Gaussian Processes](https://distill.pub/2019/visual-exploration-gaussian-processes/)
* [Gaussian Processes for Machine Learning](http://www.gaussianprocess.org/gpml/chapters/RW.pdf)
* [Automatic Model Construction with Gaussian Processes](https://www.cs.toronto.edu/~duvenaud/thesis.pdf)

# ¡Gracias! 🤓

### Raúl Peralta Lozada
* Blog: https://raulpl.github.io/
* Twitter: @tnak25
* email: raulpl25@gmail.com
* OPI Analytics